In [ ]:
!pip install -q kaggle


In [ ]:
!mkdir ~/.kaggle

In [ ]:
!mv .kaggle /root/

mv: cannot stat '.kaggle': No such file or directory


In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download arbethi/diabetic-retinopathy-level-detection

100% 9.65G/9.66G [01:28<00:00, 166MB/s]
100% 9.66G/9.66G [01:28<00:00, 117MB/s]


In [ ]:
!unzip yes.zip

Archive:  /content/yes.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/yes.zip or
        /content/yes.zip.zip, and cannot find /content/yes.zip.ZIP, period.


In [ ]:
imageSize = [299, 299]
#trainPath = "input/diabetic-retinopathy-level-detection/preprocessed dataset/preprocessed dataset/training"
#testPath = "input/diabetic-retinopathy-level-detection/preprocessed dataset/preprocessed dataset/testing"
trainPath = "../content/preprocessed dataset/preprocessed dataset/training"
testPath = "../content/preprocessed dataset/preprocessed dataset/testing"

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.xception import Xception, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import h5py

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, vertical_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
type(train_datagen)

keras.preprocessing.image.ImageDataGenerator

In [ ]:
training_set =train_datagen.flow_from_directory(trainPath, target_size = (299,299),batch_size = 32, class_mode = 'categorical')
test_set = test_datagen.flow_from_directory(testPath, target_size = (299,299),batch_size = 32, class_mode = 'categorical')

Found 3662 images belonging to 5 classes.
Found 734 images belonging to 5 classes.


In [ ]:
xception = Xception(input_shape = imageSize + [3], weights = 'imagenet', include_top = False)

In [ ]:
for layer in xception.layers:
  layer.trainable = False

In [ ]:
x = Flatten()(xception.output)

In [ ]:
prediction = Dense(5, activation = 'softmax')(x)

In [ ]:
model = Model(inputs = xception.input, outputs = prediction)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
r = model.fit_generator(training_set, validation_data = test_set, epochs = 100, steps_per_epoch = len(training_set)//32, validation_steps = len(test_set)//32)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/100
1/3 [=========>....................] - ETA: 40s - loss: 1.6129 - accuracy: 0.0938

KeyboardInterrupt: ignored

In [ ]:
tf.__version__

'2.9.2'

In [ ]:
model.save('final.h5')